In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# General epidemic simulation with super-spreaders, extinguishing events, quarantine, and multiple outbreak centers
 by Roger Ison, roger@miximum.info

  Outbreaks are represented with a reasonably standard "SIR" (Susceptible, Infected, Recovered) model.
The idea is that you have an initial population of uninfected but susceptible individuals, and a small
number of initially infected individuals. Disease spreads from each infected indivdidual to some number
of susceptible ones at a rate R0. If R0 > 1 the outbreak will explode; if < 1 it will extinguish. 
The simulation proceeds in cycles of fixed duration. Some number of infected cases die; the rest recover.
Recovered individuals can't be re-infected.

  Note that R0 is the "naive population" spread rate, when almost everybody is uninfected. When herd immunity
is enabled in the model, then the spread rate is reduced according to the population proportion of uninfected indivduals.

  The model proceeds in cycles, discussed below. On each cycle, the currently infected individuals infect
some number of new cases, while going on themselves to either recover or die. If modeling multiple, simultaneous
outbreaks, the cycles of all the outbreaks are synchronized as simulated time proceeds.

## Modeling super-spreaders
  This simulation goes beyond basic exponential math by modeling "super-spreaders". While R0 is the average
transmission rate, for every infected individual we draw a spread rate from a log-normal distribution having a
mean of R0 with some specified dispersion. As a result, some infected cases spread to a large number of new victims,
while others spread virtually not at all - that is the nature of log-normal distributions.
Therefore, the outcome of a simulation is stochastic in the log-normal spread rate. The sim runs multiple 
trials with the same parameters to get an idea of the range of possible outcomes.

  See here:  https://en.wikipedia.org/wiki/Log-normal_distribution  to learn about log-normal distributions.
 
## Modeling extinction events
  The model allows to specify an "extinction event" which occurs at a specified cycle number. The 
extinction event, if present, changes the underlying spread rate R0 to a new, specified number. If it is
less than 1.0, the epidemic will eventually die out. Extinction events might include an effective medication;
introduction of a vaccine; or a partially effective quarantine.

  Of course, if the model includes herd immunity then the outbreak will also extinguish before everyone gets
infected. In this case, the extinction event need not reduce R0 below 1.0; even a small positive number like
1.1 will still yield improved outcomes.

## Modeling herd immunity
  If this feature is enabled in the model parameters, then the effective spread rate will be reduced as the
number of recovered individuals grows. The scale of reduction of R0 is the proportion of uninfected individuals
to uninfectable individuals. 

## Multiple outbreak centers
  The model supports creating multiple, communicating outbreak centers in geographically dispersed populations.
A small proportion of population is modeled to move between each pair of centers. Outbreak centers can be created 
either randomly in the area of a 2D circle around the initial, root outbreak; or along a line, with the spacing 
between centers increasing by powers of 2.

  The proportion of population who travel, or the probability of travel in each direction, is defined as an 
exponentially declining function of distance; the decay rate or scale of distahnce is a parameter of the simulation.
In the model, the proportion of individuals who travel is selected; some are infected, some are not. The infected
individuals contribute to spread where they visit; the uninfected ones may become infected where they visit, and
then return home. 

  Because the number of travelers is generally small, infected travelers are not assumed to spend
the entire cycle where they visit; they may contribute to infection both at home and abroad. This could be 
regarded as an imperfection in the simulation's bookkeeping. Moreover, no accounting is made as to whether
traveling individuals might visit more than one other outbreak center. These bookkeeping details are probably
unimportant to the final outcome. 

  The purpose of this feature is not to emulate any specific geographical map, but to observe the delays 
between outbreaks, and other pandemic behaviors, and also the effects of regional quarantines (which are not
as effective as one might hope).

## Modeling travel quarantine
  The model allows imposition of a strict quarantine that blocks travel between geographically separated
outbreak centers. This is intended to help understand the effect of such policies. The quarantine does NOT
operate within individual outbreak centers; use the extinction events for that.
  
  By playing with the simulation's basic parameters, you can get a numerical sense of how a real epidemic might 
play out. If you adjust them to track reported case numbers, you can estimate how long it might be before burn-out,
how many cases and deaths may occur, and the impact of a vaccine coming available.

# Structure of the simulation

  Each outbreak is represented by an "Outbreak" class object. Being Python objects, outbreaks execute slowly compared
to compiled code. An outbreak begins with a fixed population of 10 million unless you edit the code. One outbreak is the ROOT, 
where initial infection occurs. All other outbreaks start with no infections. 

  Each step of an outbreak is implemented by calling the OneCycle() method. OneCycle() does the bookkeeping of generating
new infections from the current active infections, and recovering or killing off the current active infections.
The number of individuals in the uninfected, infected, recovered, and dead pools is recorded after each cycle step.
For multi-center outbreaks, each cycle has two phases. First, the number of travelers is determined, both infected
and uninfected. These are recorded in Outbreak object local variables. Then OneCycle() determines
the new numbers of individuals in the SIR pools. During multi-center simulations, OneCycle() calls other methods to
pass traveler infection consequences back to the various Outbreak objects.

  This architecture is a compromise between agent-based modeling and simple differential equations. An agent based
model would have many more parameters and distributions, such as the likelihood that one agent will encounter another
during a cycle. The more parameters a model has, the harder it is to "tune" to the real world, and the harder it is
to understand. In the end, an agent model must be run millions of times over a range of parameters, to understand its
expected behavior and variability.

  The model here only deals at the individual level in one place: where, for every infected individual, the model
draws from a log-normal distribution to determine the number of people that person infects for the next cycle. This is
fast and gives a comprehensible and intellectually defensible result.


In [3]:
# This section provides the log-normal super-spreader distribution behavior 
# and implements the Outbreak object class.

import math
import numpy as np
import pandas as pd
import copy

def Whole(f):
    return (int)(np.round(f))

def getLogRate(desiredR0, logDispersion):
    # if you want a generated log-normal distribution with desiredR0 mean spread rate, this is the underlying generator mean to use
    return math.log(desiredR0) - logDispersion * logDispersion / 2.0

def estMean(logR0, dispersion):
    # estimate mean spread rate from numpy log-normal distribution parameters.
    # reminder, numpy lognormal params determine the underlying Gaussian distribution, which gets exponentiated!
    return np.mean(np.random.lognormal(logR0, dispersion, size=200000))

def estSigma(logR0, dispersion):
    # estimate standard deviation of the generated log-normal R0 distribution.
    # reminder, numpy lognormal params are for the underlying Gaussian distribution, which gets exponentiated!
    # the standard deviation produced by this process will be a bit larger than exp(disp)
    return np.std(np.random.lognormal(logR0, dispersion, size=200000))

class Outbreak:
    # I habitually "declare" things, due to previous language experiences. Of course in Python it isn't necessary.
    identifier = None
    state = None
    herdImmunity = False
    params: {}
    maxCycles = 100
    initialInfections = 100
    geoParams = None
    geoLoc = (0.0, 0.0)
    
    nName = []
    nWeek = []
    nInfected = []
    nRecovered = []
    nDead = []
    nTotal = []
    nUninfected = []
    nPercent = []
    save = {}
    
    # the following are object-scope state variables, so  we can externally run the sim one step at a time
    population = None
    r0 = None
    dispersion = None
    logR0 = None
    approxR0 = None
    deathRate = None
    uninfectedPool = None
    priorUninfectedPool = None
    infectedPool = None
    finishedPool = None
    deadPool = None
    cycleIncomingInfectors = None
    cycleReturningInfectors = None
    conditionalProbGetInfected = None
    noTravel = None
    

    def SaveCycle(self, pop, week, infected, recovered, died, uninfected):
        # save the data from one cycle step
        self.nWeek.append(week)
        self.nName.append(self.identifier)
        self.nInfected.append(Whole(infected))
        self.nRecovered.append(Whole(recovered))
        self.nDead.append(Whole(died))
        self.nTotal.append(self.nInfected[-1] + self.nRecovered[-1] + self.nDead[-1])
        self.nUninfected.append(Whole(uninfected))
        self.nPercent.append(np.around(100.0 * (infected/pop), decimals=3))
                
    def OneCycle(self, cycle):
        # run one step of the simulation, passing in the cycle number
        if cycle == self.params["extinguishOnCycle"] :
            self.logR0 = getLogRate(self.params["extinguishingR0"], self.dispersion)
        week = cycle * self.params["infectiousWeeks"]
        deaths = self.infectedPool * self.deathRate
        self.deadPool = self.deadPool + deaths
        self.finishedPool = self.finishedPool + self.infectedPool - deaths
        priorUninfectedPool = self.uninfectedPool
        # generate lognormal distribution of transmissions for current infected population
        intInfected = Whole(self.infectedPool + self.cycleIncomingInfectors + self.cycleReturningInfectors)
        spreaders = np.random.lognormal(self.logR0, self.dispersion, size=intInfected)
        if self.herdImmunity:
            spreaders = spreaders * (self.uninfectedPool / self.population)
        # if this outbreak is part of a pandemic simulation, global quarantine may be imposed
        qcycle = -1
        if not (self.geoParams is None):    # pandemic simulation
            qcycle = self.geoParams["quarantineOnCycle"]
        self.noTravel = self.noTravel or (qcycle >= 0 and cycle >= qcycle)
        self.infectedPool = np.sum(spreaders)
        self.infectedPool = min(self.infectedPool, self.uninfectedPool)
        self.uninfectedPool = self.uninfectedPool - self.infectedPool
        # send travelers home!
        self.uninfectedPool = max(0.0, self.uninfectedPool - self.cycleIncomingInfectors - self.cycleReturningInfectors)
        # approximate probability of getting infected using this center's overall population bucket counts
        # this estimate is used to process travelers in pandemics
        self.conditionalProbGetInfected = 0.0
        if priorUninfectedPool != 0.0:
            self.conditionalProbGetInfected = self.infectedPool / priorUninfectedPool
        self.SaveCycle(self.population, week, self.infectedPool, self.finishedPool, self.deadPool, self.uninfectedPool)
        # zero out accumulators in case this is a multi-center epidemic
        self.cycleIncomingInfectors = 0.0
        self.cycleReturningInfectors = 0.0
        if self.uninfectedPool <= 0:
            # don't save the last cycle, the world has already ended
            self.state = "collapsed"
        elif self.infectedPool <= 1.0:
            self.state = "extinguished"
        else:
            self.state = "continue"
        
    def ReceiveForeignInfectors(self, numForeign):
        # infected foreigners arrive but they will propagate according to this center's herd immunity
        if not self.noTravel:
            self.cycleIncomingInfectors += numForeign
    
    def ReceiveReturningInfectors(self, numReturning):
        # some of this center's uninfected travelers return from the distant center having been infected
        # they will be moved from the uninfected pool to the infected pool, and then infect others in this center according to its herd immunity
        if not self.noTravel:
            self.cycleReturningInfectors += numReturning
    
    def InfectForeignVisitors(self, numForeign):
        # numForeign visitors arrive, and some become infected according to this center's herd immunity
        # but how to estimate that number?
        if self.noTravel:
            nVisitorsInfected = 0.0
        else:
            nVisitorsInfected = numForeign * self.conditionalProbGetInfected
        #print("{} visited by {} resulting in {} infections".format(self.identifier, Whole(numForeign), Whole(nVisitorsInfected)))
        return nVisitorsInfected
    
    def Simulate(self):
        # run a single, complete stepwise simulation until it terminates
        # this is not what you do to simulate multi-center epidemics
        for cycle in range(self.maxCycles):
            self.OneCycle(cycle)
            if self.infectedPool < 1.0:
                break
        # call Summarize() to get the results, esp. if want to print them
    
    def Summarize(self, printHist=True):
        if printHist:
            if self.state == "extinguished":
                print("Outbreak {} extinguished".format(self.identifier))
            elif self.state == "collapsed":
                print("Outbreak {} burned out".format(self.identifier))
            elif self.state == "continue":
                print("Outbreak {} ongoing".format(self.identifier))
            df = pd.DataFrame(self.save)
            print(df)
            print("\n")   
        maxInfected = np.max(self.nInfected)
        maxTotal = np.max(self.nTotal)
        maxDead = np.max(self.nDead)
        return {"Weeks":self.nWeek[-1], "MaxActive":maxInfected, "TotalCases":maxTotal, "TotalDead":maxDead}
    
    def __init__(self, ident, params, geographic=None, location=(0.0, 0.0), infect=0):
        self.identifier = ident
        self.herdImmunity = params["herd"]
        self.geoParams = geographic
        self.geoLoc = location
        self.initialInfections = infect
        self.params = copy.deepcopy(params)
        self.population = self.params["population"]
        # create lists that will track epidemic history
        self.nWeek = []
        self.nName = []
        self.nInfected = []
        self.nRecovered = []
        self.nDead = []
        self.nTotal = []
        self.nUninfected = []
        self.nPercent = []
        self.save = {"Name":self.nName, "Week":self.nWeek, "Active":self.nInfected, "Active%":self.nPercent, "Recovered":self.nRecovered, "Died":self.nDead, "Total":self.nTotal, "Uninfected":self.nUninfected}
        # initialize variable that will reflect the epidemic's numerical state
        self.r0 = self.params["R0"]
        self.dispersion = self.params["dispersion"]
        # find a base rate such that LogNormal gives desired dispersion of spreads with desired average spread rate
        self.logR0 = getLogRate(self.r0, self.dispersion)
        self.deathRate = self.params["deathRate"]
        self.uninfectedPool = self.population
        self.priorUninfectedPool = self.population
        self.infectedPool = self.initialInfections          # initial infections to get started
        self.finishedPool = 0.00
        self.deadPool = 0.0
        self.noTravel = False
        self.cycleIncomingInfectors = 0.0
        self.cycleReturningInfectors = 0.0
        self.conditionalProbGetInfected = 0.0
        # this object is ready to go
        self.state = "continue"

# end class Outbreak


In [4]:
# demonstrate log-normal stats

desiredDisp = 0.80     # 0.80 is a reasonable number for Wuhan
desiredR0 = 2.5        # estimated average spread rate
sampleSize = 100000
logRate = getLogRate(desiredR0, desiredDisp)
print("Desired generated distribution mean R0 ={:5.2f} with underlying generator dispersion ={:5.2f}".format(desiredR0, desiredDisp))
print("  underlying log-rate for generator = {}".format(logRate))
superspreaders = np.random.lognormal(logRate, desiredDisp, size=sampleSize)
superspreaders = np.sort(superspreaders)
tot = np.sum(superspreaders)
eightyPct = tot * 0.8
eightyPctCount = 0
accum = 0
for ii in range(sampleSize) :
    accum += superspreaders[-ii]
    eightyPctCount += 1
    if accum > eightyPct:
        break
pctOfRange = 100.0 * eightyPctCount / sampleSize
average = np.mean(superspreaders)
median = np.median(superspreaders)
sigma = np.std(superspreaders)
smallest = np.min(superspreaders)
largest = np.max(superspreaders)

print("It's stochastic, but for one generated, log-normal example with 100,000 incidents:")
print("  mean spread rate =   {:10.7f}".format(average))
print("  median spread rate = {:10.7f}".format(median))
print("  std deviation of spread rate distribution = {:10.7f}".format(sigma))
print("  spread rate range = [{:4.3f} .. {:4.3f}]".format(smallest, largest))
print("  80% of transmissions are generated by the top {:3.1f}% of spreaders".format(pctOfRange))




Desired generated distribution mean R0 = 2.50 with underlying generator dispersion = 0.80
  underlying log-rate for generator = 0.596290731874155
It's stochastic, but for one generated, log-normal example with 100,000 incidents:
  mean spread rate =    2.5000082
  median spread rate =  1.8144444
  std deviation of spread rate distribution =  2.3755830
  spread rate range = [0.075 .. 50.690]
  80% of transmissions are generated by the top 51.7% of spreaders


In [20]:
# Here are some single-outbreak experiments to try

# This is probably what happened in Wuhan City / Hubei province: the outbreak ran to burnout.
# While China did impose both isolation and inter-center quarantine, it was too late there.
Wuhan =  \
         {"Experiment": "Wuhan/Hubei Run to burnout",
          "population": 10.0E6,        # 10 million ~ population of Wuhan
          "herd": True,                # herd immunity effect?
          "R0": 2.5,                   # mean disease transmission rate for infected individuals
          "dispersion": 0.80,          # dispersion of underlying normal distribution for log-normal R0
          "deathRate": 0.025,          # let's hope not this bad
          "infectiousWeeks": 1,        # 1 week per infectious cycle
          "extinguishOnCycle": -1,     # -1 means no extinguishing event
          "extinguishingR0": 0.70}     # when extinguishing event occurs, spread rate falls to this number

# This is what happens if isolation is imposed very early in an outbreak:
EarlyStringentIsolation = \
         {"Experiment": "Early, stringent isolation",
          "population": 10.0E6,        # 10 million ~ population of Wuhan
          "herd": True,                # herd immunity effect?
          "R0": 2.5,                   # mean disease transmission rate for infected individuals
          "dispersion": 0.80,          # dispersion of underlying normal distribution for log-normal R0
          "deathRate": 0.025,          # let's hope not this bad
          "infectiousWeeks": 1,        # 1 week per infectious cycle
          "extinguishOnCycle": 3,      # -1 means no extinguishing event
          "extinguishingR0": 0.70}     # when extinguishing event occurs, spread rate falls to this number

# This is what happens if the isolation is imposed too late and is not completly stringent
LateSloppyIsolation = \
         {"Experiment": "Late, sloppy isolation",
          "population": 10.0E6,        # 10 million ~ population of Wuhan
          "herd": True,                # herd immunity effect?
          "R0": 2.5,                   # mean disease transmission rate for infected individuals
          "dispersion": 0.80,          # dispersion of underlying normal distribution for log-normal R0
          "deathRate": 0.025,          # let's hope not this bad
          "infectiousWeeks": 1,        # 1 week per infectious cycle
          "extinguishOnCycle": 12,     # -1 means no extinguishing event
          "extinguishingR0": 1.50}     # when extinguishing event occurs, spread rate falls to this number

# This is what happens if isolation is imposed early but is not completely stringent
EarlySloppyIsolation = \
         {"Experiment": "Early, sloppy isolation",
          "population": 10.0E6,        # 10 million ~ population of Wuhan
          "herd": True,                # herd immunity effect?
          "R0": 2.5,                   # mean disease transmission rate for infected individuals
          "dispersion": 0.80,          # dispersion of underlying normal distribution for log-normal R0
          "deathRate": 0.025,          # let's hope not this bad
          "infectiousWeeks": 1,        # 1 week per infectious cycle
          "extinguishOnCycle": 12,      # -1 means no extinguishing event
          "extinguishingR0": 1.10}     # when extinguishing event occurs, spread rate falls to this number


# Here's what happens if an extinguishing event occurs mid-outbreak.
# I've set the extinguishingR0 pretty low, e.g. if an effective drug became available
# The outbreak is bad, but catastrophe is avoided.
LateExtinguishing = \
         {"Experiment": "Extinguishing event: Effective antiviral drug mid-outbeak",
          "population": 10.0E6,        # 10 million ~ population of Wuhan
          "herd": True,                # herd immunity effect?
          "R0": 2.5,                   # mean disease transmission rate for infected individuals
          "dispersion": 0.80,          # dispersion of underlying normal distribution for log-normal R0
          "deathRate": 0.025,          # let's hope not this bad
          "infectiousWeeks": 1,        # 1 week per infectious cycle
          "extinguishOnCycle": 8,      # -1 means no extinguishing event
          "extinguishingR0": 0.70}     # when extinguishing event occurs, spread rate falls to this number

# Change this model to play with the parameters yourself!
PlayWithMe = \
         {"Experiment": "Playing with parameters",
          "population": 10.0E6,        # 10 million ~ population of Wuhan
          "herd": True,                # herd immunity effect?
          "R0": 2.5,                   # mean disease transmission rate for infected individuals
          "dispersion": 0.80,          # dispersion of underlying normal distribution for log-normal R0
          "deathRate": 0.020,          # let's hope not this bad
          "infectiousWeeks": 1,        # 1 week per infectious cycle
          "extinguishOnCycle": 10,      # -1 means no extinguishing event
          "extinguishingR0": 1.10}     # when extinguishing event occurs, spread rate falls to this number

def PrintParams(params):
    pop = params["population"]             # initial population
    print(params["Experiment"])
    print("  {} million population".format(Whole(pop/1.0E6)))
    print("  Infection cycle time {} weeks".format(params["infectiousWeeks"]))
    given_logR0 = getLogRate(params["R0"], params["dispersion"])
    print("  Naive mean spread rate R0 is {:4.2f} new infections per infected individual".format(params["R0"]))
    print("  Standard deviation of the generated log-normal R0 distribution ~ {:4.2f}".format(estSigma(given_logR0, params["dispersion"])))
    print("  Standard deviation of the underlying Gaussian distribution is {:4.2f}".format(params["dispersion"]))
    print("  Death rate {:4.2f}%".format(100.0 * params["deathRate"]))
    if params["extinguishOnCycle"] >= 0:
        print("  Extinguishing event will begin on cycle {}".format(params["extinguishOnCycle"]))
        print("  Spread rate {:4.2f} after extinguishing event".format(params["extinguishingR0"]))
    else:
        print("  No extinguishing event will occur")
    if params["herd"]:
        print("  Herd immunity effect included")
    else:
        print("  No herd immunity effect")
    print()
    

import copy
import plotly.express as px

def PlotOutbreaks(obList):
    # create a tidy dict of all the data including outbreak identifier column
    all = None
    root = None
    for epi in obList:
        hist = epi.save
        if all is None:
            root = epi
            all = copy.deepcopy(hist)
        else:
            for key in hist.keys():
                all[key].extend(copy.deepcopy(hist[key]))
                
    parms = root.params
    gp = root.geoParams
    note = ""
    if not (gp is None):
        qcycle = gp["quarantineOnCycle"]
        if qcycle >= 0:
            note = " (global quarantine on cycle {})".format(qcycle)
        else:
            note = " (no global quarantine)"
    df = pd.DataFrame(all)

    fig = px.line(df, x="Week", y="Active", color='Name', title="Active infections - " + parms["Experiment"] + note)
    fig.show()
    fig = px.line(df, x="Week", y="Total", color='Name', title="Cumulative infections - " + parms["Experiment"] + note)
    fig.show()
    fig = px.line(df, x="Week", y="Died", color='Name', title="Cumulative deaths - " + parms["Experiment"] + note)
    fig.show()


In [25]:
# This section will run multiple instances of a single outbreak and prints details about the outcome
# This will give you a feeling for the range of outcomes possible due to super-spreaders

params = LateExtinguishing

PrintParams(params)
numTrials = 10
printTables = False                    # set true to see the actual cycle data for each trial
pop = params["population"]             # initial population

outbreakList = []
print("Will run {} trials\n".format(numTrials))
history = {"MaxActive":[], "TotalCases":[], "TotalDead":[]}
for trial in range(numTrials):
    epi = Outbreak("Outbreak{:02d}".format(trial+1), params, infect=100)
    outbreakList.append(epi)
    epi.Simulate()
    outcome = epi.Summarize(printTables)
    for key in history.keys():
        history[key].append(outcome[key])

mostActive, leastActive = np.max(history["MaxActive"]), np.min(history["MaxActive"])
mostCases, leastCases = np.max(history["TotalCases"]), np.min(history["TotalCases"])
mostDeaths, leastDeaths = np.max(history["TotalDead"]), np.min(history["TotalDead"])

if printTables:
    PrintParams(params)
    
print("Best trial, per {} million population:".format(Whole(pop/1.0E6)))
print("  {:10,d} was the largest number of concurrent active infections, or {:4.2f}% of population".format(Whole(leastActive), 100 * (leastActive / pop)))
print("  {:10,d} cases occurred by end of simulation, or {:4.1f}% of population".format(leastCases, 100 * (leastCases / pop)))
print("  {:10,d} died, or {:4.3f}% of population".format(leastDeaths, 100 * (leastDeaths / pop)))

print("\nWorst trial, per {} million population:".format(Whole(pop/1.0E6)))
print("  {:10,d} was the largest number of concurrent active infections, or {:4.2f}% of population".format(Whole(mostActive), 100 * (mostActive / pop)))
print("  {:10,d} cases occurred by end of simulation, or {:4.1f}% of population".format(mostCases, 100 * (mostCases / pop)))
print("  {:10,d} died, or {:4.3f}% of population".format(mostDeaths, 100 * (mostDeaths / pop)))

print("\nRatio of worst/best cases = {:4.2f}".format(mostDeaths / leastDeaths))

PlotOutbreaks(outbreakList)


Extinguishing event: Effective antiviral drug mid-outbeak
  10 million population
  Infection cycle time 1 weeks
  Naive mean spread rate R0 is 2.50 new infections per infected individual
  Standard deviation of the generated log-normal R0 distribution ~ 2.36
  Standard deviation of the underlying Gaussian distribution is 0.80
  Death rate 2.50%
  Extinguishing event will begin on cycle 8
  Spread rate 0.70 after extinguishing event
  Herd immunity effect included

Will run 10 trials

Best trial, per 10 million population:
     114,154 was the largest number of concurrent active infections, or 1.14% of population
     432,665 cases occurred by end of simulation, or  4.3% of population
      10,817 died, or 0.108% of population

Worst trial, per 10 million population:
     179,063 was the largest number of concurrent active infections, or 1.79% of population
     661,636 cases occurred by end of simulation, or  6.6% of population
      16,541 died, or 0.165% of population

Ratio of wors

In [26]:
# Model geographic dispersion of coupled outbreaks 
import numpy as np
import math
import pandas as pd

centers = []                              # list of the outbreaks including ROOT
travelers = None                          # will be a square numpy array of travel probabilities between pairs of centers

def CreateCenters(gp, ep):
    # create the array of population centers, each is an Outbreak object
    others = gp["numOthers"]
    radius = gp["radius"]
    # generate random (r,theta) to locate the outbreaks within the geographic circle
    radii = np.random.uniform(0.0, radius, size=others)
    thetas = np.random.uniform(0.0, 2.0 * np.pi, size=others)
    # convert to (x,y) locations
    xpos = np.cos(thetas) * radii
    ypos = np.sin(thetas) * radii
    coords = [xy for xy in zip(xpos, ypos)]
    # root has some initial cases
    cents = [Outbreak("ROOT", ep, gp, infect=10)]
    # others are initially uninfected
    cents.extend([Outbreak("C{:03d}".format(k+1), ep, gp, coords[k], infect=0) for k in range(others)])
    return cents

def CreateCentersInALine(gp,ep):
    coords = [(math.pow(2.0, j), 0.0) for j in range(6,12)]
    cents = [Outbreak("ROOT", ep, gp, infect=10)]
    cents.extend([Outbreak("C{:03d}".format(k+1), ep, gp, coords[k], infect=0) for k in range(len(coords))])
    return cents

def Distance(a, b):
    # Cartesian distance between two outbreak centers
    if a is b:
        return 0.0
    return math.sqrt( (a.geoLoc[0] - b.geoLoc[0])**2 + (a.geoLoc[1] - b.geoLoc[1])**2 )

def ProportionWhoTravel(distance, gp):
    # exponential decay of probability of travel as a function of distance
    factor = -1.0 / gp["distanceDecay"]
    return math.exp(factor * distance)

def CreateMap(epiParams, geoParams, layout="circular"):
    # create the population centers
    global centers, travelers
    
    if geoParams["layout"] == "circular":
        # centers randomly distributed in a circular area around root
        centers = CreateCenters(geoParams, epiParams)
    else:
        # centers on a line with inverse binary increasing distances between them
        centers = CreateCentersInALine(geoParams, epiParams)
    
    numCenters = len(centers)
    print("Locations of other population centers")
    for aCenter in centers:
        if aCenter is centers[0]:
            continue
        miles = Distance(centers[0], aCenter)
        print("{} {:8.0f} miles from ROOT at {}   ".format(aCenter.identifier, miles, aCenter.geoLoc))
    # assign probability of travel between centers as a function of distance
    travelers = np.zeros((numCenters, numCenters))
    distances = np.zeros((numCenters, numCenters), dtype=np.int64)
    for irow in range(numCenters):
        for jcol in range(numCenters):
            d = Distance(centers[irow], centers[jcol])
            distances[irow, jcol] = Whole(d)
            travelers[irow, jcol] = ProportionWhoTravel(d, geoParams)
    # pretty-print the probabilities. 
    # there must be some Python package with better control over formatting! But what?
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', -1)
    pd.set_option('display.width', 250)
    pd.set_option('precision', 7)
    
    df = pd.DataFrame(distances)
    print("\nDistances between centers")
    print(df)
    print()

    seeProbs = np.round(travelers, 6)
    df = pd.DataFrame(seeProbs)
    
    print("\nProbability of travel between centers")
    print(df)
    print()
    
def TimeToStop(centers):
    # multi-center simulation stops unless at least one center has 1 or more active infections
    stop = True
    for epi in centers:
        if epi.infectedPool >= 1.0 :
            stop = False
            break
    return stop

def Briefly(centers):
    # tokenized view of current state of each outbreak object, call on each cycle to get a summary string
    brief = []
    for epi in centers:
        if epi.state == "extinguished" or epi.infectedPool < 1.0:
            brief.append("x")
        elif epi.state == "continue":
            brief.append("c")
        else:
            brief.append("b")
    return "".join(brief)
    
def Pandemic(epiParams, geoParams, cycles=500):
    global centers, travelers
    #send travelers between centers
    qos = geoParams["quarantineOnCycle"]
    for cycle in range(cycles):
        # NB:  self.cycleIncomingInfectors and self.cycleReturningInfectors are zeroed out each time, at end of OneCycle() method
        # so they can accumulate traveler counts for the next cycle
        leapTotal = 0.0
        for irow in range(len(centers)):
            source = centers[irow]
            for icol in range(len(centers)):
                sink = centers[icol]
                if sink is source:
                    continue
                # get probability of travel from source to sink
                proportion =  travelers[irow, icol]
                if qos >= 0 and cycle >= qos:
                    proportion = 0.0
                outgoingInfectors = source.infectedPool * proportion
                sink.ReceiveForeignInfectors(outgoingInfectors)
                outgoingUninfected = source.uninfectedPool * proportion
                returningInfectors = sink.InfectForeignVisitors(outgoingUninfected)
                source.ReceiveReturningInfectors(returningInfectors)
                leapTotal = leapTotal + outgoingInfectors + returningInfectors
        print("Cycle {:4d} {:10d} travelers  {}".format(cycle,Whole(leapTotal), Briefly(centers)))
        # run the cycle
        for epi in centers:
            epi.OneCycle(cycle)
        if TimeToStop(centers):
            break
    # after running the simulation, print the history of each center
    print()
    for epi in centers:
        epi.Summarize(False)


In [27]:
# run multi-center epidemic

# geographic parameters for pandemic model
geoParams = {"layout": "circular",        # "circular" or "linear"
             "radius": 1000.0,            # centers randomly in a circle around ROOT outbreak
             "numOthers": 9,             # number of centers simulated in addition to the ROOT where disease starts
             "distanceDecay": 80,         # probability of travel between centers = 
                                          #  exp(-1.0*distance/distanceDecay)
             "quarantineOnCycle": 10}     # -1 if no global quarantine

# epidemic parameters
epiParams = LateExtinguishing
PrintParams(epiParams)
if geoParams["quarantineOnCycle"] < 0:
    print("No global travel quarantine will be imposed")
else:
    print("Global travel quarantine will be imposed at cycle {}\n".format(geoParams["quarantineOnCycle"]))

CreateMap(epiParams, geoParams)
Pandemic(epiParams, geoParams, cycles=300)
PlotOutbreaks(centers)


Extinguishing event: Effective antiviral drug mid-outbeak
  10 million population
  Infection cycle time 1 weeks
  Naive mean spread rate R0 is 2.50 new infections per infected individual
  Standard deviation of the generated log-normal R0 distribution ~ 2.36
  Standard deviation of the underlying Gaussian distribution is 0.80
  Death rate 2.50%
  Extinguishing event will begin on cycle 8
  Spread rate 0.70 after extinguishing event
  Herd immunity effect included

Global travel quarantine will be imposed at cycle 10

Locations of other population centers
C001      594 miles from ROOT at (-296.39969817725364, -515.1363568486878)   
C002      700 miles from ROOT at (559.8090093698554, 419.73065691353696)   
C003      309 miles from ROOT at (-56.85687301311161, -304.15429527864876)   
C004      517 miles from ROOT at (-195.02297271471883, 478.836460787761)   
C005      785 miles from ROOT at (-192.074699936122, 761.3293841547855)   
C006      867 miles from ROOT at (845.5092578044332, 19